In [ ]:
# 4) IBIs from your Bangle PPG peaks, then compare to PolarHRM
import numpy as np

# 4a) Convert peak indices to times (s) and then to IBIs (ms)
t_sec = bangle_data["timestamp_ms"].to_numpy() / 1000.0
peak_times = t_sec[peaks]
ibi_ppg = np.diff(peak_times) * 1000  # IBI in ms

# 4b) Basic stats on PPG IBIs
sdnn_ppg  = np.std(ibi_ppg,  ddof=1)
rmssd_ppg = np.sqrt(np.mean(np.diff(ibi_ppg)**2))
pnn50_ppg = np.sum(np.abs(np.diff(ibi_ppg)) > 50) / len(ibi_ppg) * 100

print(f"PPG-IBI SDNN: {sdnn_ppg:.2f} ms")
print(f"PPG-IBI RMSSD: {rmssd_ppg:.2f} ms")
print(f"PPG-IBI pNN50: {pnn50_ppg:.2f} %")

# 4c) Side-by-side comparison to PolarHRM
print(f"Polar-HRM  SDNN: {np.std(rr, ddof=1):.2f} ms")
print(f"Polar-HRM  RMSSD: {rmssd:.2f} ms")
print(f"Polar-HRM  pNN50: {pnn50:.2f} %")

In [ ]:
# 5) PSD of interpolated RR series (Welch’s method)
import scipy.integrate
from scipy.signal import welch

# 5a) Interpolate uneven IBIs to a uniform 4 Hz grid
t_rr     = np.cumsum(rris) / 1000.0              # Polar timebase (s)
t_uniform = np.arange(t_rr[0], t_rr[-1], 1/4)  # 4 Hz
rr_interp = np.interp(t_uniform, t_rr, rris)

# 5b) Compute PSD
f, psd = welch(rr_interp, fs=4.0, nperseg=256)

# 5c) Integrate LF (0.04–0.15 Hz) & HF (0.15–0.4 Hz)
lf_mask = (f >= 0.04) & (f < 0.15)
hf_mask = (f >= 0.15) & (f < 0.4)
lf_power = scipy.integrate.trapezoid(psd[lf_mask], f[lf_mask])
hf_power = scipy.integrate.trapezoid(psd[hf_mask], f[hf_mask])

print(f"LF power (0.04–0.15 Hz): {lf_power:.1f}")
print(f"HF power (0.15–0.4 Hz): {hf_power:.1f}")
print(f"LF/HF ratio:      {lf_power/hf_power:.2f}")

In [ ]:
# 6) Poincaré plot and SD1/SD2
import matplotlib.pyplot as plt

# 6a) Build scatter of successive IBIs
x = ibi_ppg[:-1]
y = ibi_ppg[1:]

plt.figure(figsize=(5,5))
plt.scatter(x, y, s=10, alpha=0.6)
plt.title("Poincaré Plot (IBIₙ vs. IBIₙ₊₁)")
plt.xlabel("IBIₙ (ms)")
plt.ylabel("IBIₙ₊₁ (ms)")
plt.axis('equal')
plt.show()

# 6b) Compute SD1 & SD2
diffs = y - x
sd1 = np.sqrt(np.var(diffs) / 2)
sd2 = np.sqrt(2*np.var(ibi_ppg) - np.var(diffs) / 2)

print(f"SD1 (short-term variability): {sd1:.2f} ms")
print(f"SD2 (long-term variability):  {sd2:.2f} ms")
print(f"SD1/SD2 ratio:                {sd1/sd2:.2f}")